In [1]:
import requests
import urllib
import json
from typing import List, Dict
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from datetime import datetime, timedelta
import urllib3

In [2]:
def send_email(sender_email, receiver_email, subject, body, smtp_server, smtp_port, password, attachment_path):
    # Create a multipart message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the body to the message
    msg.attach(MIMEText(body, 'plain'))

    # Attach the file
    try:
        with open(attachment_path, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename={attachment_path.split("/")[-1]}')
            msg.attach(part)
    except Exception as e:
        print(f"Could not attach the file: {e}")
        return

    try:
        # Create a secure SSL context and connect to the SMTP server
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, password)  # Login to the email account
            server.sendmail(sender_email, receiver_email, msg.as_string())  # Send email
            print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")

In [3]:
def make_captcha_request(anchor,varChr,varVh,varBg):
    anchorr = str(anchor)
    anchorr = anchorr.strip()
    keysite = anchorr.split('k=')[1].split("&")[0]
    var_co = anchorr.split("co=")[1].split("&")[0]
    var_v = anchorr.split("v=")[1].split("&")[0]

    r1 = requests.get(anchorr).text

    token1 = r1.split('recaptcha-token" value="')[1].split('">')[0]

    var_chr = str(varChr)
    var_vh = str(varVh)
    var_bg = str(varBg)
    var_chr = str(urllib.parse.quote(var_chr))
    print("\n\nBypassing Recaptcha...")

    payload = {
        "v":var_v,
        "reason":"q",
        "c":token1,
        "k":keysite,
        "co":var_co,
        "hl":"en",
        "size":"invisible",
        "chr":var_chr,
        "vh":var_vh,
        "bg":var_bg
    }

    r2 = requests.post("https://www.google.com/recaptcha/api2/reload?k={}".format(keysite), data=payload)
    try:
        token2 = str(r2.text.split('"rresp","')[1].split('"')[0])
    except:
        token2 = 'null'

    if token2 == "null":
        print("\nRecaptcha not vulnerable : \n\n"+str(r2.text))
        
    else:
        print("\nRecaptcha Bypassed : \n\n"+str(token2))
       
        returnanchor=anchorr
        returnrelod=f"https://www.google.com/recaptcha/api2/reload?k={keysite}"
        returnpayload=f"v={var_v}&reason=q&c=<token>&k={keysite}&co={var_co}&hl=en&size=invisible&chr={var_chr}&vh={var_vh}&bg={var_bg}"
            
        return returnanchor,returnrelod,returnpayload
    

In [4]:

def generateresponse(anchorurl, reloadurl, payload):
    s = requests.Session()
    r1 = s.get(anchorurl).text
    token1 = r1.split('recaptcha-token" value="')[1].split('">')[0]
    payload = payload.replace("<token>", str(token1))
    r2 = s.post(reloadurl, data=payload, headers={"Content-Type": "application/x-www-form-urlencoded"})
    try:
        token2 = str(r2.text.split('"rresp","')[1].split('"')[0])
        return token2
    except:
        return ""
    

In [5]:
anchorresult,reloadurlresult,payloadresult=make_captcha_request('https://www.google.com/recaptcha/enterprise/anchor?ar=1&k=6Le5HB8qAAAAACVrDjBtWwnN3Ry5W1zCoV8EZoCj&co=aHR0cHM6Ly9wLmRvbmFyeS5jb206NDQz&hl=en&v=EGbODne6buzpTnWrrBprcfAY&size=invisible&cb=mqhzibkq69xi','',' 6009931042','!XFqgWl8KAAQeBu1XbQEHewF3r1uHI2K6ioroh_8HdshF71OdCl_0vnUsr-qyUE10ZEWjb1fCAv0LouH4kuXjiiXDFrqSLfTtFreWo6uV426VFvd5MCviCasLbH2YDKGpv6jmLk7NF_wJIku84MwyGF2Cdxg4TBATPN50ZYOudlADShMS9CuXpKZ-VC45Ggs_JeDRga0qHGpmqyJAHzwl9BLSeCqieRgtMnSS7oDzJ5vN8jKBjKEHkAWKF-W7sZVQ5NlFATCkv61-poyB_6QqCmqBYGFF2NTikI6HNiR9A-JTwc3InKXygJFr5_vrddsFXrGRAX0no1SWeHUt7uT1juwraHeueD2dDKBkH22_D8coA12A-CfvtdOh1aEjIdWD7L9t3DksdYW5hjN6VoahPGdJctCc4mYQNUxuuCsX4ddzGONH-opywCM3Z_SKs866t7leSfRor9_nnc4tZqGUX118Ij5bIAb4evGrRbi8p1o0-YWNWdUPYU1rDCSXZPZB28i1ENZ11xTGnARUy77eogCECHFreyXShCjHp87rqwRwj0vDcvk6aRs9Xd1EouLwL1iegUh1-xjXIyjUu2zjwreTc7KpssIg0nEeoxMt1WhmKqo3AsC2iOy3ScvSv7Kwb5lKJaTzEcjcpcoHLWykk90J-C-Z1sEGaCFZ-VFZoSNLceWEKKVubS_iWPeHQGXkXHduMSc1osY4bLx6MoDc_4H8C6X8IxEUcYvRAqkAIoiTDsoYBBgFxOWFQ-E4bbfHz2_wnhBntUx3OWqxp4wWYSq6UTt79nNXHQ-r8WmWUd6_pxtV3fhnF3BrHJ8xJwn_46v6giTAzZyhWSm1_Cwbc2PO4G4l0LEk-g5z7Y333BlIXJ7v1O1wxRsJpFVVGBvKTt48rkMgvrsoiklkJoHRDunpYjuIVpQl-8eTbpK4eunQ6i-hTy5dkPDRCXHiQTnrNWOuEjNhIxfLK86jc1XdETdm5ZxLkAMQ7QcvF-V2FY_4AwLQhKeNWT0r80YYcX3qgMGF4UvW4EMWLGxjh5PIvuWUW40In9r-i48-vgmQe8xEY4kn8yhNhe9eWjYDf2U1Ibzqf2FCRta1gR0-zQhQa40yXW_xTrRR6kMJnb_5SSbZ8BGWr1O00RRNkoYOv5cBdNIZLq_586UsCMRCHt_Bfi8wy-lcBCskMnXsl-XGclOYJ7RN0NIKVsXQycvd6WuWSjKk7ET31XZtkSDL-3VRwx1qP8k4be23oY_NgG4kcd8XcnlvXbf5g0EnJJWxKFqQGc3eM9HpDKZrQzSodziPWXYVB96aLcPx7neaQ_XRlNKa8klHjJnqt1dAtfvMFzqEcAvW0i-PX0bzaoxvILlkMYLNPIucK5McUrxCI_OdpPHN6FJjDqfzjLhofVFYe9pdeSVKDJpz6uEoGSzcawFUXTVvN2_fWRFQ8Jr-0uOBMSgc8Rr2G1LJxbtz-EFzDv6xEbvQgaIWvQvwU9dR-QsJZe4A8kc7ciQ96chifCgaJ5tOdtbELddABgC9mVGwRGVQ_aXxvwm84F2PjkcyDI-LQMFqFxz36qaO28CQ-KhigujZE9w77_sBBKrbjtMKX-MADNYpYUQyHUI7eXAWIOtpOrFlSnaGwPOtJRO6ohhJbK2cUPevOva9QrVvnYPOOYkloXKlIZNhhCikHJRP_8k6361N0E3NCe_XewdhO1_XMydkAIL6BKNO1s_JzUrdo0fYlgZvgRoCuqTBilP-bwNXFh80IvsC9DH6AJwKt4fx3Qc3V0u8coxE8z9biHbD6KKCBP8pUfoZd9EnMEL1U-OB48lK1TENzy_u6_KsAKfOlD8oebo8NMiurlfhiRtkUHVrPT5qPxRC-iACQAU1THIwr7PNwTwXI3rmnxEk9riXYefT0nzaJ5X9v-hI0upcQ_t7TE6_laNh3Rto0B3ESRa5haT1QQol7NgquPH3h2LwGdPF4ncmNV04tZYb6R9WxHWJ7cA-xN1r1hZ9pFMbk9gIiQ*')




Bypassing Recaptcha...

Recaptcha Bypassed : 

03AFcWeA7_xuc3e0ty005Gp1v1y4bpL55QGex2xeRJyVwIejnsJMbJ4-RTIwNHCOiqu0pHaY3C5SoDK2aQHaITqF582Go2n8y-FUXWKSgk7XBqrv7v837qI4ZqUnX5NEMcOhMouL3Gl_CCZJNEvQTSet6pxsCh83bfjSb8COdxIJoMnHhMxFbwGoCCgFQs7ENA7NkAoGqW9d7NMZKHqWIqVvyc3_VkpNLlcyHPKr-Yh0kfshnWBFw6g8yDZJAHc0WzwsTBomEW5ksDQprAFxYLE3ba2x5h2xb5fj6FKlHGWXxJUeWlYi7LX43Vir9431yrBRIufB9uJ8-VxJTiByH6DAT1DahEn_PBsIHxnK9CzS2FnkIkyObDybQZ7l4YIhPB-7Hr0e5AFhW2jyrxqTZFtdplOUjD5cLG8nU0GBXupY4kSyL1M-AU-9lWu1xnBDU3vQ2NimIDPV2zYcJcgsd5tyN2qokOdqCzyTbOybbgZpH5JXg-fV2Ti_MAVoryTQPGr2rEnYBtY0uI8WadARkPwa4PM0Kt_ziPpBxKCKam9Lx-V3k6Uny67mzdY0GKCQI-C1IJT71AL_q8HC-bMvem3gdZBUzaBQecDPxkq6vsFDBzobLS6kG4_28LLGtpn_AZCSqjnZ4rMvm3xn_RQvd5PyDWUHI4Vj5LxR3zGS7Jt5v_REtH_pNWS5fLHoThjYZVmFCaD_5bSl_zEj65FwfHNX1xatleCp7Lewj66Zjwf71ovjsceC7ekzIos3U3Y_8oImCatWGirWmIvl1DBf-fOSmV8XikTUjO7oO4bWp5Fej2deU92ec_i-F-CZdCjQ_ZnuXWGKufZAyeD7ir4lRUBZLp5sgVpEjFIbYrSovPt2P2F3w9OUWyDbOV5O-ipVD1YHjfeS96ZT_TvMqBpc6IShCf2Su6IkUkxFzKsep8EV3TzupGV4ilO

In [9]:
respnsetoken=generateresponse(anchorresult,reloadurlresult,payloadresult)



In [ ]:


def login_function(email,password,response_token):
# API URL
    url = "https://webapi.donary.com/v2/authentication/login"

    # Payload for login, replace with actual email, password, and generated reCAPTCHA token
    payload = {
        "email":email ,
        "password": password,
        "recapcha": response_token  
    }

    # Headers
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    # Make the POST request
    response3 = requests.post(url, json=payload, headers=headers)

    # Check the response status and content
    if response3.status_code == 200:
        print("Login successful!")
        return response3  # This will show the server's response
    else:
        print("Login failed. Status code:", response3.status_code)
        print("Response:", response3.text)  # Print the error message from the server
        


login_response=login_function(str(""),str( ''   ),respnsetoken)


Login successful!


In [8]:
login_response.json()['accessToken']

'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJVc2VySWQiOiIyNjQ2IiwiRXZlbnRHdUlkIjoiMTJiOGU4NjMtOWRjMC00ZDhjLThmMDEtNTk2OGNkNTY0ZDljIiwiTWFjQWRkcmVzcyI6IjQwYWE5NmQ3LWMxMjItNDhlMC05YTRiLTZmNzA3MGY2MzAzYyIsIlJlZnJlc2hUb2tlbiI6Ijh6SExma3krZTZ0ai9iVVRIY2hIeG5idkk4Nk1HaDlWbEh5VHNqYTA2L1E9IiwiQWNjZXNzVG9rZW5FeHBpcnkiOiIwOS8yNy8yNCAxMToyMDoxNyIsIkV2ZW50Q3VycmVuY3kiOiJVU0QiLCJVc2VyVW5pcXVlSUQiOiJiNDNkZGJhZi1iMzkyLTQ3YjUtYjEzNy05MjUwOGUyZDdhYTkiLCJDYW1wYWlnbklkIjoiIiwiSXNUb2lyZW1DYW1wYWlnbkxvZ2luIjoiRmFsc2UiLCJUb2lyZW1Vc2VyQWNjb3VudElkIjoiIiwiSXNEZXZpY2VMb2dpbiI6IkZhbHNlIiwibmJmIjoxNzI3MzkyODE3LCJleHAiOjE3Mjc0MzYwMTcsImlhdCI6MTcyNzM5MjgxNywiaXNzIjoiaXNzdWVyIiwiYXVkIjoiYXVkaWVuY2UifQ.qRh-2wjH6y55zwn-IKatZu3Uj8e_-KM0Ygb_yzhFrGo'

In [ ]:
url = "https://webapi.donary.com/v1/paymenttransaction/GetPaymentTrans"
today_date = datetime.now().strftime('%Y-%m-%d')
yesterday_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')

tomorrow_date = (datetime.now() + timedelta(days=3)).strftime('%Y-%m-%d')
"12b8e863-9dc0-4d8c-8f01-5968cd564d9c"

payload = {
    "eventGuId": '',
    "fromDate": '2000-09-26',
    "toDate": today_date,
}

# Define the headers, including the Bearer token
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {login_response.json()['accessToken']} "
}


response = requests.post(url, headers=headers, json=payload)

In [14]:
from collections import Counter

In [28]:
data = response.json()

# Extract the list of payments from 'paymentTransGridModel'
data

{'paymentTransGridModel': [{'paymentId': 15529450,
   'paymentType': 'Credit Card',
   'paymentTypeId': 4,
   'device': 'DM_1631',
   'receiptNum': '1470',
   'paymentDateTime': '2024-10-09T00:00:00',
   'refNum': 'Visa-6340',
   'approval': 'Success',
   'amount': 180.0,
   'accountId': 2213711,
   'accountNum': 'DEFAULT',
   'fullName': None,
   'fullNameJewish': None,
   'phoneLabels': None,
   'phoneNumbers': None,
   'emailLabels': None,
   'emailAddresses': None,
   'colectorName': "ר' יצחק וואסערמאן",
   'collectorId': 1903013,
   'reasonId': None,
   'reasonName': None,
   'locationId': 1132,
   'locationName': 'פאפא',
   'campaignId': None,
   'campaignName': '',
   'paymentStatusId': 1,
   'paymentStatus': 'Success',
   'createdDate': '2024-10-09T09:41:49.0033333',
   'scheduleID': None,
   'scheduleNum': None,
   'scheduleInfo': None,
   'cardHolderName': 'JUDA ROSENFELD',
   'address': None,
   'cityStateZip': None,
   'group': None,
   'class': None,
   'father': None,
   

AttributeError: 'dict' object has no attribute 'paymentTransGridModel'

In [30]:


# Extract the list of payments from 'paymentTransGridModel'
payments = data.get('paymentTransGridModel', [])
print(len(payments[0]))

# Extract all sourceTypeId values from the payments
source_type_ids = [item['paymentStatus'] for item in payments if 'paymentType' in item]
# print(  source_type_ids)

# Count occurrences of each unique sourceTypeId
source_type_count = Counter(source_type_ids)

# Convert to list of tuples (sourceTypeId, count)
source_type_list = list(source_type_count.items())

# Print the result
print(payments[0].keys())

101
dict_keys(['paymentId', 'paymentType', 'paymentTypeId', 'device', 'receiptNum', 'paymentDateTime', 'refNum', 'approval', 'amount', 'accountId', 'accountNum', 'fullName', 'fullNameJewish', 'phoneLabels', 'phoneNumbers', 'emailLabels', 'emailAddresses', 'colectorName', 'collectorId', 'reasonId', 'reasonName', 'locationId', 'locationName', 'campaignId', 'campaignName', 'paymentStatusId', 'paymentStatus', 'createdDate', 'scheduleID', 'scheduleNum', 'scheduleInfo', 'cardHolderName', 'address', 'cityStateZip', 'group', 'class', 'father', 'fatherInLaw', 'walletId', 'note', 'smsSent', 'emailSent', 'batchNum', 'mailSent', 'printSent', 'deviceId', 'reasonNum', 'paymentJewishDate', 'globalId', 'advancedFieldNames', 'advancedFieldValues', 'reasons', 'campaigns', 'reasonIds', 'campaignIds', 'sourceTypeId', 'defaultPhone', 'defaultPhoneLabel', 'defaultEmail', 'defaultEmailLabel', 'reasonAmounts', 'firstName', 'lastName', 'title', 'titleJewish', 'firstNameJewish', 'lastNameJewish', 'suffixJewish'

: 

In [27]:
def get_items(login_response):

    url = "https://webapi.donary.com/v1/paymenttransaction/GetPaymentTrans"
    today_date = datetime.now().strftime('%Y-%m-%d')

    payload = {
        "eventGuId": "12b8e863-9dc0-4d8c-8f01-5968cd564d9c",
        "fromDate": '2024-09-25',
        "toDate": today_date,
    }

    # Define the headers, including the Bearer token
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {login_response.json()['accessToken']} "
    }
    

    # Send the POST request to fetch the transactions
    try:
    
        response = requests.post(url, headers=headers, json=payload)
    except Exception as e:
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        response= requests.post(url, headers=headers, json=payload,verify=False)
        print(e)
        

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON data
        transactions = response.json()
        
        # Process the transactions data as needed
        return transactions  # Pretty print the JSON data
    else:
        
        return 'Failed'
        
transactions_results=get_items(login_response)

In [24]:
transactions_results

{'paymentTransGridModel': [{'paymentId': 15218542,
   'paymentType': 'Credit Card',
   'paymentTypeId': 4,
   'device': 'DM_2050',
   'receiptNum': '1059',
   'paymentDateTime': '2024-09-26T00:00:00',
   'refNum': 'Master-3686',
   'approval': 'Success',
   'amount': 50.0,
   'accountId': 5273443,
   'accountNum': '83452',
   'fullName': 'Meir Hersh Zilberstein',
   'fullNameJewish': 'מו"ה מאיר הערש זילבערשטיין הי"ו',
   'phoneLabels': 'Cell',
   'phoneNumbers': '8452130625',
   'emailLabels': 'Home',
   'emailAddresses': 'meirsilb@gmail.com',
   'colectorName': 'מענדיל וואלף שפירא',
   'collectorId': 5273492,
   'reasonId': None,
   'reasonName': None,
   'locationId': None,
   'locationName': None,
   'campaignId': '157323',
   'campaignName': 'Project338C',
   'paymentStatusId': 1,
   'paymentStatus': 'Success',
   'createdDate': '2024-09-26T16:02:21.0433333',
   'scheduleID': None,
   'scheduleNum': None,
   'scheduleInfo': None,
   'cardHolderName': None,
   'address': '36 Cedar L

In [31]:
def filter_transactions_by_payment_type(transactions, payment_types):
    """
    Filters a list of transactions based on the payment types.

    :param transactions: List of transaction dictionaries.
    :param payment_types: A list of payment types to filter by (e.g., ['Cash', 'Pledger']).
    :return: List of transactions that match the specified payment types.
    """
    filtered_transactions = [
        transaction for transaction in transactions 
        if transaction.get('paymentType') in payment_types
    ]
    
    return filtered_transactions

Filtered_data=filter_transactions_by_payment_type(transactions_results['paymentTransGridModel'],['OJC','Pledger','Matbia','Credit Card'])

Filtered_data



[{'paymentId': 15218542,
  'paymentType': 'Credit Card',
  'paymentTypeId': 4,
  'device': 'DM_2050',
  'receiptNum': '1059',
  'paymentDateTime': '2024-09-26T00:00:00',
  'refNum': 'Master-3686',
  'approval': 'Success',
  'amount': 50.0,
  'accountId': 5273443,
  'accountNum': '83452',
  'fullName': 'Meir Hersh Zilberstein',
  'fullNameJewish': 'מו"ה מאיר הערש זילבערשטיין הי"ו',
  'phoneLabels': 'Cell',
  'phoneNumbers': '8452130625',
  'emailLabels': 'Home',
  'emailAddresses': 'meirsilb@gmail.com',
  'colectorName': 'מענדיל וואלף שפירא',
  'collectorId': 5273492,
  'reasonId': None,
  'reasonName': None,
  'locationId': None,
  'locationName': None,
  'campaignId': '157323',
  'campaignName': 'Project338C',
  'paymentStatusId': 1,
  'paymentStatus': 'Success',
  'createdDate': '2024-09-26T16:02:21.0433333',
  'scheduleID': None,
  'scheduleNum': None,
  'scheduleInfo': None,
  'cardHolderName': None,
  'address': '36 Cedar Lane #201',
  'cityStateZip': 'Monsey NY 10952',
  'group':

In [32]:
def clean_payment_data(payment_data):
    # Fields to keep
    fields_to_keep = {
        'receiptNum': 'Receipt #',
        'paymentDateTime': 'Payment Date',
        'fullNameJewish': 'Donor Jewish Name',
        'createdDate': 'Created Date & Time',
        'amount': 'Amount',
        'paymentType': 'Payment Type',
        'approval': 'Approval',
        'refNum': 'Ref #',
        'paymentStatus': 'Status',
        'scheduleNum': 'Schedule #',
        'scheduleInfo': 'Schedule Info',
        'note': 'NOTE',
        'sourceTypeId': 'Source',
        'accountNum': 'Account #',
        'fullName': 'Donor English Name',
        'address': 'Address',
        'cityStateZip': 'City State Zip',
        'defaultPhone': 'Phone Number',
        'emailAddresses': 'Email',
        'pledgePayment': 'Amount applied',
        'currencyAmount': 'Currency Amount',
        'gatewayBatchNum': 'Gateway Batch #'
    }

    # Clean the data
    cleaned_data = []
    for payment in payment_data:
        cleaned_payment = {new_key: payment[old_key] for old_key, new_key in fields_to_keep.items() if old_key in payment}
        cleaned_data.append(cleaned_payment)

    return cleaned_data

cld=clean_payment_data(Filtered_data)
cld

[{'Receipt #': '1059',
  'Payment Date': '2024-09-26T00:00:00',
  'Donor Jewish Name': 'מו"ה מאיר הערש זילבערשטיין הי"ו',
  'Created Date & Time': '2024-09-26T16:02:21.0433333',
  'Amount': 50.0,
  'Payment Type': 'Credit Card',
  'Approval': 'Success',
  'Ref #': 'Master-3686',
  'Status': 'Success',
  'Schedule #': None,
  'Schedule Info': None,
  'NOTE': '',
  'Source': 6,
  'Account #': '83452',
  'Donor English Name': 'Meir Hersh Zilberstein',
  'Address': '36 Cedar Lane #201',
  'City State Zip': 'Monsey NY 10952',
  'Phone Number': None,
  'Email': 'meirsilb@gmail.com',
  'Amount applied': 50.0,
  'Currency Amount': 50.0,
  'Gateway Batch #': '32832386'},
 {'Receipt #': '1058',
  'Payment Date': '2024-09-26T00:00:00',
  'Donor Jewish Name': 'מו"ה אפרים זאב שטערן הי"ו',
  'Created Date & Time': '2024-09-26T16:02:10.4633333',
  'Amount': 50.0,
  'Payment Type': 'Credit Card',
  'Approval': None,
  'Ref #': 'Master-3686',
  'Status': 'Error',
  'Schedule #': None,
  'Schedule Info'

In [20]:
def make_excell(data):
    t_date = datetime.now().strftime('%Y-%m-%d')
    df = pd.DataFrame(data)
    df.to_excel(f"Data for Date{t_date}.xlsx", index=False)
    return df

In [21]:
make_excell(cld)

,Receipt #,Payment Date,Donor Jewish Name,Created Date & Time,Amount,Payment Type,Approval,Ref #,Status,Schedule #,...,Source,Account #,Donor English Name,Address,City State Zip,Phone Number,Email,Amount applied,Currency Amount,Gateway Batch #
0,1469,2024-08-08T00:00:00,None,2024-08-08T10:24:06.7233333,18.0,Credit Card,Success,Master-2739,Success,None,...,6,DEFAULT,None,None,None,0000000000,None,NaN,18.0,31580032
1,1468,2024-04-01T00:00:00,Rodrigo Lima,2024-07-29T17:47:55.15,1.0,Credit Card,None,None,Error,None,...,12,DNR1171,Rodrigo Lima,________________________,________________________,None,None,NaN,1.0,None
2,1467,2024-04-01T00:00:00,Rodrigo Lima,2024-07-29T17:45:34.2966667,1.0,Credit Card,None,None,Error,None,...,12,DNR1170,Rodrigo Lima,________________________,________________________,None,None,NaN,1.0,None
3,1466,2024-04-01T00:00:00,Rodrigo Lima,2024-07-29T17:43:56.1,1.0,Credit Card,None,None,Error,None,...,12,DNR1169,Rodrigo Lima,________________________,________________________,None,None,NaN,1.0,None
4,1465,2024-04-01T00:00:00,Rodrigo Lima,2024-07-29T17:28:39.6933333,1.0,Credit Card,None,None,Error,None,...,12,DNR1168,Rodrigo Lima,________________________,________________________,None,None,NaN,1.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,1007,2022-05-08T00:00:00,אליעזר איינהארן,2022-05-08T18:57:14.9266667,36.0,Credit Card,Success,Amex-1049,Success,None,...,11,325165,elizer einhorn,,,8457079333,eliezereinhorn@gmail.com,36.0,NaN,None
441,1006,2022-05-08T00:00:00,אליעזר איינהארן,2022-05-08T18:56:28.9166667,36.0,Credit Card,Error,Amex-1049,Error,None,...,11,325165,elizer einhorn,,,8457079333,eliezereinhorn@gmail.com,NaN,NaN,None
442,1005,2022-05-06T00:00:00,ר' ישעי גאנץ,2022-05-06T12:54:36.2033333,44.0,Credit Card,Success,Master-9645,Success,None,...,7,319969,R' Yeshaya GANZ,4 Wiener Drive 211,Monsey NY 10952,8453526493,shayagantz@gmail.com,44.0,NaN,None
443,1003,2022-05-05T00:00:00,הערשל גאנץ,2022-05-05T16:38:05.6966667,72.0,Credit Card,Success,Visa-6079,Success,None,...,11,4280987347,Hershel Gantz,"8 האדמו""ר מויז'ניץ 8",בני ברק 5161022,972583291664,heregantz@gmail.com,72.0,NaN,None


In [44]:
def generateresponse(anchorurl, reloadurl, payload,email,receveemail,password):
    s = requests.Session()
   
    r1 = s.get(anchorurl).text

    
         
    token1 = r1.split('recaptcha-token" value="')[1].split('">')[0]
    payload = payload.replace("<token>", str(token1))
    

    r2 = s.post(reloadurl, data=payload, headers={"Content-Type": "application/x-www-form-urlencoded"})

    
    
    
    

    token2 = str(r2.text.split('"rresp","')[1].split('"')[0])
    return token2
   

In [46]:




# Usage
send_email(
    sender_email="rdan99848@gmail.com",
    receiver_email="plptest2002@gmail.com",
    subject="Daily File",
    body="A copy of the requested file",
    smtp_server="smtp.gmail.com",  # For Gmail
    smtp_port=587,  # For Gmail
    password="grzk kisi qhhx iicc",  # Use an app password if necessary
    attachment_path=r"D:\needed\dns projects\donary\payments.xlsx"  # Replace with the path to your file
)


Email sent successfully!
